In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv("student_mat.csv", sep=";")

In [3]:
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [4]:
df = data[["G1", "G2", "G3", "studytime", "failures", "absences", "freetime", "health"]]

In [5]:
df.head()

,G1,G2,G3,studytime,failures,absences,freetime,health
0,5,6,6,2,0,6,3,3
1,5,5,6,2,0,4,3,3
2,7,8,10,2,3,10,3,3
3,15,14,15,3,0,2,2,5
4,6,10,10,2,0,4,3,5


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   G1         395 non-null    int64
 1   G2         395 non-null    int64
 2   G3         395 non-null    int64
 3   studytime  395 non-null    int64
 4   failures   395 non-null    int64
 5   absences   395 non-null    int64
 6   freetime   395 non-null    int64
 7   health     395 non-null    int64
dtypes: int64(8)
memory usage: 24.8 KB


### predict G3 marks 

In [7]:
predict = "G3"

In [8]:
# y = mx + c

# x will have g1 and g2 value only 
X = np.array(df.drop([predict], 1)) 
print(X)

[[ 5  6  2 ...  6  3  3]
 [ 5  5  2 ...  4  3  3]
 [ 7  8  2 ... 10  3  3]
 ...
 [10  8  1 ...  3  5  3]
 [11 12  1 ...  0  4  5]
 [ 8  9  1 ...  5  2  5]]


In [9]:
# y will have g3 value which we will predict
y = np.array(df[[predict]])
print(y)

[[ 6]
 [ 6]
 [10]
 [15]
 [10]
 [15]
 [11]
 [ 6]
 [19]
 [15]
 [ 9]
 [12]
 [14]
 [11]
 [16]
 [14]
 [14]
 [10]
 [ 5]
 [10]
 [15]
 [15]
 [16]
 [12]
 [ 8]
 [ 8]
 [11]
 [15]
 [11]
 [11]
 [12]
 [17]
 [16]
 [12]
 [15]
 [ 6]
 [18]
 [15]
 [11]
 [13]
 [11]
 [12]
 [18]
 [11]
 [ 9]
 [ 6]
 [11]
 [20]
 [14]
 [ 7]
 [13]
 [13]
 [10]
 [11]
 [13]
 [10]
 [15]
 [15]
 [ 9]
 [16]
 [11]
 [11]
 [ 9]
 [ 9]
 [10]
 [15]
 [12]
 [ 6]
 [ 8]
 [16]
 [15]
 [10]
 [ 5]
 [14]
 [11]
 [10]
 [10]
 [11]
 [10]
 [ 5]
 [12]
 [11]
 [ 6]
 [15]
 [10]
 [ 8]
 [ 6]
 [14]
 [10]
 [ 7]
 [ 8]
 [18]
 [ 6]
 [10]
 [14]
 [10]
 [15]
 [10]
 [14]
 [ 8]
 [ 5]
 [17]
 [14]
 [ 6]
 [18]
 [11]
 [ 8]
 [18]
 [13]
 [16]
 [19]
 [10]
 [13]
 [19]
 [ 9]
 [16]
 [14]
 [13]
 [ 8]
 [13]
 [15]
 [15]
 [13]
 [13]
 [ 8]
 [12]
 [11]
 [ 9]
 [ 0]
 [18]
 [ 0]
 [ 0]
 [12]
 [11]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [12]
 [15]
 [ 0]
 [ 9]
 [11]
 [13]
 [ 0]
 [11]
 [ 0]
 [11]
 [ 0]
 [10]
 [ 0]
 [14]
 [10]
 [ 0]
 [12]
 [ 8]
 [13]
 [10]
 [15]
 [12]
 [ 0]
 [ 7]
 [ 0]
 [10]
 [ 7]
 [12]
 [10

In [10]:
# spliting the data into test and train in the ratio of 90:10
# spliting into four arrays
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.1, random_state=0)



In [11]:
linear = linear_model.LinearRegression()

In [12]:
linear.fit(x_train, y_train)
acc = linear.score(x_test, y_test)
# accuracy 
print(acc)

0.8157637810976065


In [13]:
print('Coefficient(m): \n', linear.coef_)
print('Intercept: \n', linear.intercept_)

Coefficient(m): 
 [[ 0.12180565  1.00111514 -0.14128604 -0.33959899  0.0445597   0.07745965
   0.09278119]]
Intercept: 
 [-2.04612398]


In [14]:
prediction = linear.predict(x_test)

In [15]:
prediction

array([[14.03265159],
       [10.57843158],
       [ 9.73474248],
       [15.12866074],
       [16.22417569],
       [11.90495387],
       [12.75097236],
       [10.84994865],
       [ 8.63869528],
       [14.91357684],
       [ 7.20600741],
       [14.12705682],
       [10.49995806],
       [ 6.96257345],
       [14.88504943],
       [14.31888545],
       [13.35360439],
       [ 9.14814275],
       [ 9.11873838],
       [14.8265731 ],
       [ 5.85852828],
       [ 3.13370637],
       [ 5.99396721],
       [14.87134651],
       [11.34160401],
       [16.46060749],
       [ 9.37488641],
       [20.13751833],
       [-0.43383496],
       [12.9196671 ],
       [-1.82343414],
       [ 6.53192062],
       [ 7.43422753],
       [ 5.11256405],
       [12.72789984],
       [10.19478109],
       [12.63766173],
       [18.96084844],
       [ 8.30779402],
       [11.73453568]])

In [20]:
print(y_test)

[[14]
 [10]
 [ 9]
 [15]
 [16]
 [12]
 [14]
 [11]
 [ 9]
 [15]
 [10]
 [11]
 [10]
 [ 8]
 [15]
 [14]
 [12]
 [ 0]
 [10]
 [16]
 [ 0]
 [ 0]
 [ 8]
 [16]
 [12]
 [15]
 [10]
 [19]
 [ 0]
 [14]
 [ 0]
 [10]
 [ 8]
 [ 4]
 [13]
 [10]
 [13]
 [18]
 [ 8]
 [11]]


In [24]:
for x in range(len(prediction)):
    # 
    print(x_test[x],y_test[x], prediction[x])
    

[14 14  3  0  4  3  4] [14] [14.03265159]
[11 11  3  0  0  3  5] [10] [10.57843158]
[ 9 10  3  0  9  3  5] [9] [9.73474248]
[16 15  2  0  2  4  1] [15] [15.12866074]
[15 16  2  0  2  2  5] [16] [16.22417569]
[13 12  3  0  6  3  3] [12] [11.90495387]
[12 13  2  0  0  3  4] [14] [12.75097236]
[10 11  2  0  6  4  4] [11] [10.84994865]
[8 9 1 0 5 2 5] [9] [8.63869528]
[16 15  3  0  0  3  2] [15] [14.91357684]
[8 8 3 0 2 3 4] [10] [7.20600741]
[11 12  2  0 54  3  5] [11] [14.12705682]
[11 11  4  0  8  4  1] [10] [10.49995806]
[7 8 4 0 8 1 3] [8] [6.96257345]
[14 15  2  0  2  4  1] [15] [14.88504943]
[14 14  1  0  2  3  5] [14] [14.31888545]
[10 13  1  0 12  4  5] [12] [13.35360439]
[ 9 10  3  0  0  3  3] [0] [9.14814275]
[10  9  2  0 10  4  5] [10] [9.11873838]
[14 15  2  0  0  2  3] [16] [14.8265731]
[6 7 2 0 0 1 4] [0] [5.85852828]
[6 5 1 3 0 3 4] [0] [3.13370637]
[7 7 3 0 0 3 4] [8] [5.99396721]
[13 15  2  0  2  3  3] [16] [14.87134651]
[12 12  4  0  1  2  3] [12] [11.34160401]
[15 16  3